## LINCS MOA Classifcation on LINCS curated using support vector machine

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing  import StandardScaler
from sklearn.model_selection import train_test_split
#from sklearn.svm import SVC
from sklearn import svm
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score

In [2]:
file_prostate=pd.read_hdf("/opt/raid10/genomics/rashid/GCN/data/GSE92742_fully_restricted.hdf")

In [3]:
df_prostate=file_prostate.reset_index(level=[0,1,2,3,4,5,6]) #change from multi index to single index

In [4]:
prostate_y=df_prostate["moa"].values #change moa column to numpy array
df_prostate= df_prostate.drop(columns=["pert_id","pert_name","cell_id","primary_site","subtype","moa","Fold"])
prostate_X=df_prostate.values #get the matrix of feature values for 978 genes


In [5]:
labels,uniques=pd.factorize(prostate_y) #labels are the encodings
prostate_y=labels

In [6]:
sc=StandardScaler()
prostate_X=sc.fit_transform(prostate_X) #normalize feature data

In [7]:
#SPLIT DATA
X_train,X_test,y_train,y_test=train_test_split(prostate_X,prostate_y,test_size=0.2,random_state=42)#split into train and test

In [8]:
clm=svm.SVC()
clm.fit(X_train,y_train) #fit SVM with training data. took around 10 mins on mac


/home/rashid/.local/lib/python3.5/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [9]:
pred_clm=clm.predict(X_test) #see how the test data is predicted
#METRICS
print(confusion_matrix(y_test,pred_clm))
print(classification_report(y_test,pred_clm))
acc_clm=accuracy_score(y_test,pred_clm) #result for svm
print(acc_clm) #print precision score 
#results matches in the paper where accuracy is 66.61, macro F1 is 0.43. Took 4-5 minutes on macbook pro 2.7 GHz inter core i5 processor, 16 GB 1867 MHz DDR3 Memomry, Intel Iris Graphics 6100 1536 MB Graphics

[[   0    4    0 ...   19  512    0]
 [   0    8    1 ...   23  587    0]
 [   0    2   16 ...   30  548    0]
 ...
 [   0    0    0 ... 2699  729    3]
 [   0    0    0 ...  170 4984    0]
 [   0    0    0 ...  685   84   62]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       583
           1       0.16      0.01      0.02       710
           2       0.67      0.03      0.05       634
           3       0.68      0.28      0.40       453
           4       0.00      0.00      0.00       205
           5       0.00      0.00      0.00       588
           6       1.00      0.09      0.17       212
           7       0.43      0.01      0.03       441
           8       0.93      0.71      0.81       859
           9       1.00      0.00      0.00       433
          10       0.00      0.00      0.00       337
          11       0.00      0.00      0.00       290
          12       0.50      0.00      0.01       487
          13   

/home/rashid/.local/lib/python3.5/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [10]:
from sklearn.externals import joblib
joblib.dump(clm,'SVM_MOA_FullCurated_notTuned')

['SVM_MOA_FullCurated_notTuned']